# <span style="color:green;">SCRAPEANDO EN STEAM:</span>

In [1]:
import requests #Importamos request para obtener información de nuestro link de interés.
url_base = "https://store.steampowered.com/" #Guardamos el url base que usaremos.

In [2]:
from bs4 import BeautifulSoup #Importamos beatifulsoup para la lectura del contenido de nuestros links.

In [3]:
import re #Importamos la librearía para el uso de expresiones regulares.
respuesta_articulo=requests.get(url_base)  #Guardamos el request del url deseado.
sopa_articulo = BeautifulSoup(respuesta_articulo.text) #Guardamos el contenido del request.
contenido =sopa_articulo.find_all("div", {"class":"button_container"}) #Guardamos en contenido las ventanas que deseamos.
print(contenido)
# Por lo que sacamos pagina principal, cuatro ventanas pero aun no estan limpias:
# noveades, ofertas , juegos gratuitos y por etiqueta.

[<div class="button_container" data-panel='{"flow-children":"row"}'>
<a class="big_button" data-panel="[]" href="https://store.steampowered.com/explore/new/?snr=1_4_4__146">
                    New Releases                </a>
<a class="big_button" data-panel="[]" href="https://store.steampowered.com/search/?specials=1&amp;snr=1_4_4__146">
                    Specials                </a>
<a class="big_button" data-panel="[]" href="https://store.steampowered.com/genre/Free to Play/?snr=1_4_4__146">
                    Free Games                </a>
<a class="big_button" data-panel="[]" href="https://store.steampowered.com/tag/browse/?snr=1_4_4__146">
                    By User Tags                </a>
</div>]


In [4]:
import re #Importamos la librearía para el uso de expresiones regulares.
x = re.findall('https[\S ]+6',str(contenido)) # obtenemos los links de interés.
print(x) 

['https://store.steampowered.com/explore/new/?snr=1_4_4__146', 'https://store.steampowered.com/search/?specials=1&amp;snr=1_4_4__146', 'https://store.steampowered.com/genre/Free to Play/?snr=1_4_4__146', 'https://store.steampowered.com/tag/browse/?snr=1_4_4__146']


## <span style="color:red;">1) Scrapeando novedades en Steam:</span>

In [5]:
def links(link): #Creamos una función para obtener el link deseado.
    return x[link]
links(0) # obtenemos el link de novedades.

'https://store.steampowered.com/explore/new/?snr=1_4_4__146'

In [6]:
respuesta_articulo=requests.get(links(0)) #Obtenemos el request del link de novedades.
sopa_articulo = BeautifulSoup(respuesta_articulo.text) #Guardamos el contenido del request.
contenido2 =sopa_articulo.find_all("div", {"class":"tab_see_more"}) #Guardamos en contenido2 las ventanas que deseamos.
x2 = re.findall('https[\S]+pop.+n',str(contenido2))  #Usamos expresiones regulares para obtener el link deseado.
print(x2)
# obtenemos el link para ver una lista de todas las novedades posibles.

['https://store.steampowered.com/search/?filter=popularnew&amp;sort_by=Released_DESC&amp;os=win']


### <span style="color:blue;">Nombres</span>

In [7]:
def nombres(cantidad): #Creamos una función con variable necesaria cantidad.
    respuesta_articulo=requests.get(x2[0]) # obtenemos el request para ver la lista de todas las novedades posibles.
    sopa_articulo = BeautifulSoup(respuesta_articulo.text)  #Guardamos el contenido del request.
    contenido3 =sopa_articulo.find_all("div", {"class":"col search_name ellipsis"}) #Guardamos en contenido3 las ventanas 
                                                                                    #que deseamos.
    lista=[] #Creamos una lista vacía.
    for i in range(cantidad): # Hacemos un for para i vaya hasta cantidad.
        lista.append(contenido3[i].text) #Añadimos a lista el contenido3[i].
    lista2=[] #Creamos una segunda lista vacía.
    for word in lista: #Hacemos un for para cada palabra de la lista.
        lista2.append(word.replace("\n",""))  #Hacemos la limpieza de datos reemplazando "/n" por "".
    return lista2 #Retormanos la lista 2.
NOMBRES=nombres(50) # la cantidad mostrada varia de acuerdo al usuario.
                    # por lo que se establece un rango minimo de 50.

### <span style="color:blue;">Plataformas</span>

In [8]:
respuesta_articulo=requests.get(x2[0]) # obtenemos el request para ver la lista de todas las novedades posibles.
sopa_articulo = BeautifulSoup(respuesta_articulo.text)  #Guardamos el contenido del request.
contenido4 =sopa_articulo.find_all("span", {"class":"platform_img"})#Guardamos en contenido4 las ventanas 
                                                                    #que deseamos.
#print(str(contenido4[0])=='<span class="platform_img win"></span>') #Con esto revisamos de que podamos usar esta igualdad.
ab=len(contenido4) #Tamaño de las filas de las plataformas totales.
a=[] #Creamos la primera lista que nos retornará los datos.
temp="" #Creamos el str para los datos temporales que se almacenarán en a.
pas = False #Creamos el pass para condicionar la adjuntación de valores.
i=0 #Contador.
while i<ab: #Mientras i sea menor a la fila de datos totales.
    if pas: #Si el pass es true.
        a.append(temp) #Guardamos en la lista a lo que tenemos en los datos temporales.
        temp = "" #Borramos los temporales.
        pas = False #Regresamos el pass a False.
    if str(contenido4[i]) == '<span class="platform_img win"></span>': #Si la fila i es windows.
        temp = temp + "Windows" #Guardamos en los temporales la palabra Windows.
        pas = True #Ponemos el pass True para borrar los temporales.
        if i<ab-1 and str(contenido4[i+1]) != '<span class="platform_img win"></span>':#Si la siguiente fila no es windows.
            pas = False #Ponemos el pass False para que siga acumulando.
    elif str(contenido4[i]) == '<span class="platform_img linux"></span>': #Si la fila i es linux.
        temp = temp + ", Linux" #Guardamos en los temporales la palabra Linux.
        pas = True #Ponemos el pass True para borrar los temporales.
    else: #Si la fila i es mac.
        temp = temp + ", Mac" #Guardamos en los temporales la palabra Mac.
        if i<ab-1 and str(contenido4[i+1]) == '<span class="platform_img win"></span>': #Si la siguiente fila es windows.
            pas = True #Ponemos el pass True para borrar los temporales.
    if i == ab-1: #si i es igual a la última fila.
        a.append(temp) #Obligamos a guardar en la lista a lo que tenemos en los datos temporales.
    i = i+1 #Subimos el i al siguiente número.
print(len(a)) #Vemos si nuestra lista a es de tamaño deseado.
plataformas = a #Guardamos lo obtenido en la variable plataformas.

50


### <span style="color:blue;">Fecha de lanzamiento</span>

In [9]:
def fecha(cantidad): #Creamos una función con variable necesaria cantidad.
    lista=[] #Creamos una lista vacía.
    for i in range(cantidad): # Hacemos un for para i vaya hasta cantidad.
        respuesta_articulo=requests.get(x2[0])  # obtenemos el request para ver la lista de todas las novedades posibles.
        sopa_articulo = BeautifulSoup(respuesta_articulo.text) #Guardamos el contenido del request.
        contenido5 =sopa_articulo.find_all("div", {"col search_released responsive_secondrow"})#Guardamos en contenido5 
                                                                                               #las ventanas que deseamos.
        lista.append(contenido5[i].text) #Guardamos en lista el contenido 5 para i.
    return lista #Retornamos la lista.
FECHA=fecha(50)

### <span style="color:blue;">Reseñas de los usuarios (Valoración, cantidad de reseñas y calificación final)</span>

In [10]:
respuesta_articulo=requests.get(x2[0]) # obtenemos el request para ver la lista de todas las novedades posibles.
sopa_articulo = BeautifulSoup(respuesta_articulo.text) #Guardamos el contenido del request.
contenido7 =sopa_articulo.find_all("span", {"class":"search_review_summary"}) #Guardamos en contenido7
                                                                              #las ventanas que deseamos.
x3 = re.findall('[0-9]+[\S ]+ive.',str(contenido7)) #Guardamos en x3 la limpieza con expresiones regulares.
valoracion=re.findall('[0-9]+%',str(x3)) #Buscamos con expresiones regulares la valoración de los usuarios.
reseñas=re.findall(' [0-9,]+',str(x3)) #Buscamos con expresiones regulares la cantidad de reseñas.
resumen=re.findall('[MVPO][A-z ]+',str(contenido7)) #Buscamos con expresiones regulares la valoración resultante.
# se extrae 3 caracteristicas importantes que se encuentran dentro del a reseña.

### <span style="color:blue;">Precio</span>

In [11]:
respuesta_articulo=requests.get(x2[0]) # Sacamos el request de la página de novedades totales.
sopa_articulo = BeautifulSoup(respuesta_articulo.text) # Lo guardamos en la variable sopa articulo el texto obtenido.
contenido6 =sopa_articulo.find_all("div", {"class":"responsive_search_name_combined"}) # Guardamos las filas de los juegos.
import re # Importamos re para hacer uso de las expresiones regulares.
x6 = re.findall('data-price-.+',str(contenido6)) # Con expresiones regulares sacamos nuestra variable objetivo.
PRECIO = re.findall('[0-9]+',str(x6)) # Obtenemos unicamente los caracteres númericos.

### <span style="color:blue;">Creando el dataframe de lo obtenido</span>

In [12]:
import pandas as pd #Importamos el paquete que nos servirá para crear el dataframe.
df = pd.DataFrame() #Creamos el dataframe en la variable df.
df["Nombre"]=NOMBRES #Creamos la columna Nombre para los nombres obtenidos.
df["Plataformas"]=plataformas #Creamos la columna Plataformas para las plataformas obtenidas.
df["Fecha de lanzamiento"]=FECHA #Creamos la columna Fecha de lanzamiento para las fechas obtenidas.
df["Valoración"]=valoracion #Creamos la columna Valoración para las valoraciones obtenidas.
df["Cantidad de reseñas"]=reseñas #Creamos la columna Cantidad de reseñas para las reseñas obtenidas.
df["Calificación final"]=resumen #Creamos la columna Calificación final para los resúmenes obtenidos.
df["Precio"]=PRECIO #Creamos la columna Precio para los precios obtenidos.

In [13]:
df['Precio'] = df['Precio'].astype(float, errors = 'raise') # hacemos la tranformación a flotante.
precioN=df["Precio"]/100 # Dividimos entre 100 el precio para que tenga sentido.
df["Precio"]=precioN #Guardamos la división realizada.
df #Vemos nuestra tabla.

,Nombre,Plataformas,Fecha de lanzamiento,Valoración,Cantidad de reseñas,Calificación final,Precio
0,ELDEN RING,Windows,"24 Feb, 2022",87%,"256,747",Very Positive,199.00
1,Counter-Strike: Global Offensive,"Windows, Mac, Linux","21 Aug, 2012",88%,"6,295,713",Very Positive,58.00
2,Apex Legends™,Windows,"4 Nov, 2020",86%,"418,742",Very Positive,0.00
3,Destiny 2,Windows,"1 Oct, 2019",84%,"454,959",Very Positive,0.00
4,MONSTER HUNTER RISE,Windows,"12 Jan, 2022",87%,"16,222",Very Positive,131.93
5,Tom Clancy's Rainbow Six® Siege,Windows,"1 Dec, 2015",87%,"876,479",Very Positive,23.99
6,FIFA 22,Windows,"30 Sep, 2021",80%,"44,007",Very Positive,65.67
7,It Takes Two,Windows,"25 Mar, 2021",96%,"68,946",Overwhelmingly Positive,55.60
8,PUBG: BATTLEGROUNDS,Windows,"21 Dec, 2017",55%,"1,985,676",Mixed,0.00
9,The Sims™ 4,Windows,"18 Jun, 2020",86%,"43,111",Very Positive,16.68


## <span style="color:red;">2) Scrapeando juegos en promociones en Steam:</span>

In [14]:
def links(link): #Creamos una función para obtener el link deseado.
    return x[link]
prom = links(1) # obtenemos el link de juegos en promociones.
prom

'https://store.steampowered.com/search/?specials=1&amp;snr=1_4_4__146'

### <span style="color:blue;">Nombres</span>

In [15]:
def nombres(cantidad): #Creamos una función con variable necesaria cantidad.
    respuesta_articulo=requests.get(prom) # obtenemos el request para ver la lista de todas las promociones.
    sopa_articulo = BeautifulSoup(respuesta_articulo.text)  #Guardamos el contenido del request.
    contenido3 =sopa_articulo.find_all("div", {"class":"col search_name ellipsis"}) #Guardamos en contenido3 las ventanas 
                                                                                    #que deseamos.
    lista=[] #Creamos una lista vacía.
    for i in range(cantidad): # Hacemos un for para i vaya hasta cantidad.
        lista.append(contenido3[i].text) #Añadimos a lista el contenido3[i].
    lista2=[] #Creamos una segunda lista vacía.
    for word in lista: #Hacemos un for para cada palabra de la lista.
        lista2.append(word.replace("\n",""))  #Hacemos la limpieza de datos reemplazando "/n" por "".
    return lista2 #Retormanos la lista 2.
NOMBRES=nombres(50) # la cantidad mostrada varia de acuerdo al usuario.
                    # por lo que se establece un rango minimo de 50.

### <span style="color:blue;">Plataformas</span>

In [16]:
respuesta_articulo=requests.get(prom) # obtenemos el request para ver la lista de todas las promociones.
sopa_articulo = BeautifulSoup(respuesta_articulo.text)  #Guardamos el contenido del request.
contenido4 =sopa_articulo.find_all("span", {"class":"platform_img"})#Guardamos en contenido4 las ventanas 
                                                                    #que deseamos.
#print(str(contenido4[0])=='<span class="platform_img win"></span>') #Con esto revisamos de que podamos usar esta igualdad.
ab=len(contenido4) #Tamaño de las filas de las plataformas totales.
a=[] #Creamos la primera lista que nos retornará los datos.
temp="" #Creamos el str para los datos temporales que se almacenarán en a.
pas = False #Creamos el pass para condicionar la adjuntación de valores.
i=0 #Contador.
while i<ab: #Mientras i sea menor a la fila de datos totales.
    if pas: #Si el pass es true.
        a.append(temp) #Guardamos en la lista a lo que tenemos en los datos temporales.
        temp = "" #Borramos los temporales.
        pas = False #Regresamos el pass a False.
    if str(contenido4[i]) == '<span class="platform_img win"></span>': #Si la fila i es windows.
        temp = temp + "Windows" #Guardamos en los temporales la palabra Windows.
        pas = True #Ponemos el pass True para borrar los temporales.
        if i<ab-1 and str(contenido4[i+1]) != '<span class="platform_img win"></span>':#Si la siguiente fila no es windows.
            pas = False #Ponemos el pass False para que siga acumulando.
    elif str(contenido4[i]) == '<span class="platform_img linux"></span>': #Si la fila i es linux.
        temp = temp + ", Linux" #Guardamos en los temporales la palabra Linux.
        pas = True #Ponemos el pass True para borrar los temporales.
    else: #Si la fila i es mac.
        temp = temp + ", Mac" #Guardamos en los temporales la palabra Mac.
        if i<ab-1 and str(contenido4[i+1]) == '<span class="platform_img win"></span>': #Si la siguiente fila es windows.
            pas = True #Ponemos el pass True para borrar los temporales.
    if i == ab-1: #si i es igual a la última fila.
        a.append(temp) #Obligamos a guardar en la lista a lo que tenemos en los datos temporales.
    i = i+1 #Subimos el i al siguiente número.
print(len(a)) #Vemos si nuestra lista a es de tamaño deseado.
plataformas = a #Guardamos lo obtenido en la variable plataformas.

50


### <span style="color:blue;">Fecha de lanzamiento</span>

In [17]:
def fecha(cantidad): #Creamos una función con variable necesaria cantidad.
    lista=[] #Creamos una lista vacía.
    for i in range(cantidad): # Hacemos un for para i vaya hasta cantidad.
        respuesta_articulo=requests.get(prom)  # obtenemos el request para ver la lista de todas las promociones.
        sopa_articulo = BeautifulSoup(respuesta_articulo.text) #Guardamos el contenido del request.
        contenido5 =sopa_articulo.find_all("div", {"col search_released responsive_secondrow"})#Guardamos en contenido5 
                                                                                               #las ventanas que deseamos.
        lista.append(contenido5[i].text) #Guardamos en lista el contenido 5 para i.
    return lista #Retornamos la lista.
FECHA=fecha(50)

### <span style="color:blue;">Reseñas de los usuarios (Valoración, cantidad de reseñas y calificación final)</span>

In [18]:
respuesta_articulo=requests.get(prom) # obtenemos el request para ver la lista de todas las promociones.
sopa_articulo = BeautifulSoup(respuesta_articulo.text) #Guardamos el contenido del request.
contenido7 =sopa_articulo.find_all("span", {"class":"search_review_summary"}) #Guardamos en contenido7
                                                                              #las ventanas que deseamos.
x3 = re.findall('[0-9]+[\S ]+ive.',str(contenido7)) #Guardamos en x3 la limpieza con expresiones regulares.
valoracion=re.findall('[0-9]+%',str(x3)) #Buscamos con expresiones regulares la valoración de los usuarios.
reseñas=re.findall(' [0-9,]+',str(x3)) #Buscamos con expresiones regulares la cantidad de reseñas.
resumen=re.findall('[MVPO][A-z ]+',str(contenido7)) #Buscamos con expresiones regulares la valoración resultante.
# se extrae 3 caracteristicas importantes que se encuentran dentro del a reseña.

### <span style="color:blue;">Precio inicial</span>

In [19]:
respuesta_articulo=requests.get(prom) # obtenemos el request para ver la lista de todas las promociones.
sopa_articulo = BeautifulSoup(respuesta_articulo.text) #Guardamos el contenido del request.
contenido5 =sopa_articulo.find_all("div", {"class":"col search_price discounted responsive_secondrow"})#Guardamos 
                                                                         #en contenido7 las ventanas que deseamos.
x7 = re.findall('strike>S..[0-9.]+',str(contenido5)) #Limpiamos un poco con expresiones regulares.
PRECIOI = re.findall('S..[0-9.]+',str(x7))  #Buscamos con expresiones regulares el precio inicial.

### <span style="color:blue;">Precio final</span>

In [20]:
respuesta_articulo=requests.get(prom) #obtenemos el request para ver la lista de todas las promociones.
sopa_articulo = BeautifulSoup(respuesta_articulo.text) #Guardamos el contenido del request.
contenido5 =sopa_articulo.find_all("div", {"class":"col search_price discounted responsive_secondrow"})#Guardamos 
                                                                         #en contenido5 las ventanas que deseamos.
PRECIOF = re.findall('S..[0-9]+.[0-9]+ ',str(contenido5)) #Buscamos con expresiones regulares el precio final.

### <span style="color:blue;">Descuento</span>

In [22]:
import re #Importamos la librearía para el uso de expresiones regulares.
respuesta_articulo=requests.get(prom) #obtenemos el request para ver la lista de todas las promociones.
sopa_articulo = BeautifulSoup(respuesta_articulo.text) #Guardamos el contenido del request.
contenido6 =sopa_articulo.find_all("span")#Guardamos en contenido6 las ventanas que deseamos
descuento = re.findall('-[0-9]+%',str(contenido6)) # #Buscamos con expresiones regulares el descuento.

### <span style="color:blue;">Creando el dataframe de lo obtenido</span>

In [23]:
import pandas as pd #Importamos el paquete que nos servirá para crear el dataframe.
df = pd.DataFrame() #Creamos el dataframe en la variable df.
df["Nombre"]=NOMBRES #Creamos la columna Nombre para los nombres obtenidos.
df["Plataformas"]=plataformas #Creamos la columna Plataformas para las plataformas obtenidas.
df["Fecha de lanzamiento"]=FECHA #Creamos la columna Fecha de lanzamiento para las fechas obtenidas.
df["Valoración"]=valoracion #Creamos la columna Valoración para las valoraciones obtenidas.
df["Cantidad de reseñas"]=reseñas #Creamos la columna Cantidad de reseñas para las reseñas obtenidas.
df["Calificación final"]=resumen #Creamos la columna Calificación final para los resúmenes obtenidos.
df["Precio inicial"]=PRECIOI #Creamos la columna Precio para los precios iniciales obtenidos.
df["Precio final"]=PRECIOF #Creamos la columna Precio para los precios finales obtenidos.
df["Descuento"]=descuento #Creamos la columna Precio para los descuentos obtenidos.
df

,Nombre,Plataformas,Fecha de lanzamiento,Valoración,Cantidad de reseñas,Calificación final,Precio inicial,Precio final,Descuento
0,The Sims™ 4,Windows,"18 Jun, 2020",86%,"43,111",Very Positive,S/.139.00,S/.16.68,-88%
1,Tom Clancy's Rainbow Six® Siege,Windows,"1 Dec, 2015",87%,"876,479",Very Positive,S/.59.99,S/.23.99,-60%
2,FIFA 22,Windows,"30 Sep, 2021",80%,"44,007",Very Positive,S/.199.00,S/.65.67,-67%
3,It Takes Two,Windows,"25 Mar, 2021",96%,"68,946",Overwhelmingly Positive,S/.139.00,S/.55.60,-60%
4,MONSTER HUNTER RISE,Windows,"12 Jan, 2022",87%,"16,222",Very Positive,S/.199.90,S/.131.93,-34%
5,Red Dead Redemption 2,Windows,"5 Dec, 2019",87%,"229,439",Very Positive,S/.199.90,S/.99.95,-50%
6,Sid Meier’s Civilization® VI,"Windows, Mac, Linux","20 Oct, 2016",83%,"147,880",Very Positive,S/.239.00,S/.35.85,-85%
7,STAR WARS Jedi: Fallen Order Deluxe Edition,Windows,"14 Nov, 2019",90%,"73,753",Very Positive,S/.169.00,S/.33.80,-80%
8,Monster Hunter: World,Windows,"9 Aug, 2018",86%,"209,584",Very Positive,S/.57.28,S/.28.64,-50%
9,Dead by Daylight,Windows,"14 Jun, 2016",81%,"419,459",Very Positive,S/.39.95,S/.19.97,-50%


## <span style="color:red;">3) Scrapeando los juegos más vendidos en Steam:</span>

In [24]:
import requests #Importamos request para obtener información de nuestro link de interés.
respuesta_articulo=requests.get(url_base) #Guardamos el request del url deseado.
sopa_articulo = BeautifulSoup(respuesta_articulo.text) #Guardamos el contenido del request.
contenido =sopa_articulo.find_all("div", {"class":"tab_see_more"}) #Guardamos en contenido las ventanas que deseamos.
print(contenido[1])
# de la pagina principal, obtenemos la vista completa de los juegos más vendidos.

<div class="tab_see_more">
							See more:							<a class="btnv6_white_transparent btn_small_tall" href="https://store.steampowered.com/search/?filter=topsellers&amp;os=win&amp;snr=1_4_4__tabsmore">
<span>Top Sellers</span>
</a>
</div>


In [25]:
import re #Importamos la librearía para el uso de expresiones regulares.
top = re.findall('https[\S ]+re',str(contenido[1])) #Usamos las expresiones regulares para obtener el link deseado.
print(top[0]) # obtenemos unicamente el link de los juegos más vendidos de steam.

https://store.steampowered.com/search/?filter=topsellers&amp;os=win&amp;snr=1_4_4__tabsmore


### <span style="color:blue;">Nombres</span>

In [26]:
def nombres(cantidad): #Creamos una función con variable necesaria cantidad.
    respuesta_articulo=requests.get(top[0]) # obtenemos el request para ver la lista de todos los juegos más vendidos.
    sopa_articulo = BeautifulSoup(respuesta_articulo.text)  #Guardamos el contenido del request.
    contenido3 =sopa_articulo.find_all("div", {"class":"col search_name ellipsis"}) #Guardamos en contenido3 las ventanas 
                                                                                    #que deseamos.
    lista=[] #Creamos una lista vacía.
    for i in range(cantidad): # Hacemos un for para i vaya hasta cantidad.
        lista.append(contenido3[i].text) #Añadimos a lista el contenido3[i].
    lista2=[] #Creamos una segunda lista vacía.
    for word in lista: #Hacemos un for para cada palabra de la lista.
        lista2.append(word.replace("\n",""))  #Hacemos la limpieza de datos reemplazando "/n" por "".
    return lista2 #Retormanos la lista 2.
NOMBRES=nombres(50) # la cantidad mostrada varia de acuerdo al usuario.
                    # por lo que se establece un rango minimo de 50.

### <span style="color:blue;">Plataformas</span>

In [27]:
respuesta_articulo=requests.get(top[0]) # obtenemos el request para ver la lista de todas todos los juegos más vendidos.
sopa_articulo = BeautifulSoup(respuesta_articulo.text)  #Guardamos el contenido del request.
contenido4 =sopa_articulo.find_all("span", {"class":"platform_img"})#Guardamos en contenido4 las ventanas 
                                                                    #que deseamos.
#print(str(contenido4[0])=='<span class="platform_img win"></span>') #Con esto revisamos de que podamos usar esta igualdad.
ab=len(contenido4) #Tamaño de las filas de las plataformas totales.
a=[] #Creamos la primera lista que nos retornará los datos.
temp="" #Creamos el str para los datos temporales que se almacenarán en a.
pas = False #Creamos el pass para condicionar la adjuntación de valores.
i=0 #Contador.
while i<ab: #Mientras i sea menor a la fila de datos totales.
    if pas: #Si el pass es true.
        a.append(temp) #Guardamos en la lista a lo que tenemos en los datos temporales.
        temp = "" #Borramos los temporales.
        pas = False #Regresamos el pass a False.
    if str(contenido4[i]) == '<span class="platform_img win"></span>': #Si la fila i es windows.
        temp = temp + "Windows" #Guardamos en los temporales la palabra Windows.
        pas = True #Ponemos el pass True para borrar los temporales.
        if i<ab-1 and str(contenido4[i+1]) != '<span class="platform_img win"></span>':#Si la siguiente fila no es windows.
            pas = False #Ponemos el pass False para que siga acumulando.
    elif str(contenido4[i]) == '<span class="platform_img linux"></span>': #Si la fila i es linux.
        temp = temp + ", Linux" #Guardamos en los temporales la palabra Linux.
        pas = True #Ponemos el pass True para borrar los temporales.
    else: #Si la fila i es mac.
        temp = temp + ", Mac" #Guardamos en los temporales la palabra Mac.
        if i<ab-1 and str(contenido4[i+1]) == '<span class="platform_img win"></span>': #Si la siguiente fila es windows.
            pas = True #Ponemos el pass True para borrar los temporales.
    if i == ab-1: #si i es igual a la última fila.
        a.append(temp) #Obligamos a guardar en la lista a lo que tenemos en los datos temporales.
    i = i+1 #Subimos el i al siguiente número.
print(len(a)) #Vemos si nuestra lista a es de tamaño deseado.
plataformas = a #Guardamos lo obtenido en la variable plataformas.

50


### <span style="color:blue;">Fecha de lanzamiento</span>

In [28]:
def fecha(cantidad): #Creamos una función con variable necesaria cantidad.
    lista=[] #Creamos una lista vacía.
    for i in range(cantidad): # Hacemos un for para i vaya hasta cantidad.
        respuesta_articulo=requests.get(top[0])  # obtenemos el request para ver la lista de todos los juegos más vendidos.
        sopa_articulo = BeautifulSoup(respuesta_articulo.text) #Guardamos el contenido del request.
        contenido5 =sopa_articulo.find_all("div", {"col search_released responsive_secondrow"})#Guardamos en contenido5 
                                                                                               #las ventanas que deseamos.
        lista.append(contenido5[i].text) #Guardamos en lista el contenido 5 para i.
    return lista #Retornamos la lista.
FECHA=fecha(50)

### <span style="color:blue;">Precio</span>

In [29]:
respuesta_articulo=requests.get(top[0]) # Sacamos el request de la página de todos los juegos más vendidos.
sopa_articulo = BeautifulSoup(respuesta_articulo.text) # Lo guardamos en la variable sopa articulo el texto obtenido
contenido6 =sopa_articulo.find_all("div", {"class":"responsive_search_name_combined"}) # Guardamos las filas de los juegos
import re # Importamos re para hacer uso de las expresiones regulares
x6 = re.findall('data-price-.+',str(contenido6)) # Con expresiones regulares sacamos nuestra variable objetivo
PRECIO = re.findall('[0-9]+',str(x6)) # Obtenemos unicamente los caracteres númericos

### <span style="color:blue;">Creando el dataframe de lo obtenido</span>

In [30]:
import pandas as pd #Importamos el paquete que nos servirá para crear el dataframe.
df = pd.DataFrame() #Creamos el dataframe en la variable df.
df["Nombre"]=NOMBRES #Creamos la columna Nombre para los nombres obtenidos.
df["Plataformas"]=plataformas #Creamos la columna Plataformas para las plataformas obtenidas.
df["Fecha de lanzamiento"]=FECHA #Creamos la columna Fecha de lanzamiento para las fechas obtenidas.
df["Precio"]=PRECIO #Creamos la columna Precio para los precios obtenidos.

In [31]:
df['Precio'] = df['Precio'].astype(float, errors = 'raise') # hacemos la tranformación a flotante.
precioN=df["Precio"]/100 # Dividimos entre 100 el precio para que tenga sentido.
df["Precio"]=precioN #Guardamos la división realizada.
df #Vemos nuestra tabla.

,Nombre,Plataformas,Fecha de lanzamiento,Precio
0,ELDEN RING,Windows,"24 Feb, 2022",199.00
1,Core Keeper,"Windows, Linux","8 Mar, 2022",28.00
2,MONSTER HUNTER RISE,Windows,"12 Jan, 2022",131.93
3,FIFA 22,Windows,"30 Sep, 2021",65.67
4,It Takes Two,Windows,"25 Mar, 2021",55.60
5,Grand Theft Auto V: Premium Edition,Windows,,49.17
6,LEGO® Star Wars™: The Skywalker Saga,Windows,"5 Apr, 2022",159.00
7,LEGO® Star Wars™: The Skywalker Saga,Windows,"5 Apr, 2022",159.00
8,NieR Replicant™ ver.1.22474487139...,Windows,"23 Apr, 2021",99.50
9,WWE 2K22,Windows,"10 Mar, 2022",239.00
